In [1]:
# Parameters
angle = 0.0


In [2]:
from ophyd import EpicsMotor, EpicsSignal
from ophyd.areadetector.plugins import PluginBase
from ophyd.areadetector import AreaDetector, ADComponent, ImagePlugin, JPEGPlugin, TIFFPlugin
from ophyd.areadetector.cam import AreaDetectorCam
from ophyd.areadetector.cam import CamBase
from bluesky import RunEngine
from bluesky.plans import scan, count
import bluesky.plan_stubs as bps
import numpy as np
from datetime import datetime
from pathlib import Path
from PIL import Image
import time, sys, os, subprocess
from collections import defaultdict

# Import configuration from external file
import config
class PvaPlugin(PluginBase):
    _suffix = 'Pva1:'
    _plugin_type = 'NDPluginPva'
    _default_read_attrs = ['enable']
    _default_configuration_attrs = ['enable']

    array_callbacks = ADComponent(EpicsSignal, 'ArrayCallbacks')

# Create RunEngine
RE = RunEngine({})

# Define the motor using config
motor = EpicsMotor(config.MOTOR_PV, name=config.MOTOR_NAME)

# Define the camera device
class MyCamera(AreaDetector):
    cam = ADComponent(AreaDetectorCam, 'cam1:') #Fixed the single camera issue?
    image = ADComponent(ImagePlugin, 'image1:')
    tiff = ADComponent(TIFFPlugin, 'TIFF1:')
    pva = ADComponent(PvaPlugin, 'Pva1:')

# Instantiate the camera using config
camera = MyCamera(config.CAMERA_PV, name=config.CAMERA_NAME)
camera.wait_for_connection()


file_path_container = []

In [3]:
# Camera configuration using config file settings

# CAM OPTIONS
camera.stage_sigs[camera.cam.acquire] = config.CAM_CONFIG['acquire']
camera.stage_sigs[camera.cam.image_mode] = config.CAM_CONFIG['image_mode'] # single multiple continuous
camera.stage_sigs[camera.cam.trigger_mode] = config.CAM_CONFIG['trigger_mode'] # internal external

# IMAGE OPTIONS
camera.stage_sigs[camera.image.enable] = config.IMAGE_CONFIG['enable'] # pva plugin
camera.stage_sigs[camera.image.queue_size] = config.IMAGE_CONFIG['queue_size']

# TIFF OPTIONS
camera.stage_sigs[camera.tiff.enable] = config.TIFF_CONFIG['enable']
camera.stage_sigs[camera.tiff.auto_save] = config.TIFF_CONFIG['auto_save']
camera.stage_sigs[camera.tiff.file_write_mode] = config.TIFF_CONFIG['file_write_mode']  # Or 'Single' works too
camera.stage_sigs[camera.tiff.nd_array_port] = config.TIFF_CONFIG['nd_array_port']  
camera.stage_sigs[camera.tiff.auto_increment] = config.TIFF_CONFIG['auto_increment']       #Doesn't work, must be ignored

# PVA OPTIONS
camera.stage_sigs[camera.pva.enable] = config.PVA_CONFIG['enable']
camera.stage_sigs[camera.pva.blocking_callbacks] = config.PVA_CONFIG['blocking_callbacks']
camera.stage_sigs[camera.pva.queue_size] = config.PVA_CONFIG['queue_size']  # or higher
camera.stage_sigs[camera.pva.nd_array_port] = config.PVA_CONFIG['nd_array_port'] 
camera.stage_sigs[camera.pva.array_callbacks] = config.PVA_CONFIG['array_callbacks']  # disable during scan


In [4]:
def wait_for_file(filepath, timeout=config.FILE_WAIT_TIMEOUT, poll_interval=config.POLL_INTERVAL):
    """Wait until a file appears on disk, or timeout."""
    start = time.time()
    while not os.path.exists(filepath):
        if time.time() - start > timeout:
            raise TimeoutError(f"Timed out waiting for file: {filepath}")
        time.sleep(poll_interval)

In [5]:
def acquire(measurement_angle):
    """Acquire image at specified angle using configuration settings"""
    # Requirements for image capturing
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    callbacks_signal = EpicsSignal(config.CALLBACKS_SIGNAL_PV, name='callbacks_signal')
    acquire_signal = EpicsSignal(config.ACQUIRE_SIGNAL_PV, name='acquire_signal')

    yield from bps.mv(callbacks_signal, 0)
    max_retries = config.MAX_RETRIES
    yield from bps.open_run()
    camera.cam.array_callbacks.put(0, wait=True)

    print("\n--- Staging camera ---")
    yield from bps.stage(camera)

    current_number = camera.tiff.file_number.get()
    
    yield from bps.mv(acquire_signal, 0)  # Triggers a single image

    # Generate filename using angle parameter
    filename = f'scan_{timestamp}_pos_{measurement_angle}_shot'           
    current_number += 1
    filepath = os.path.join(save_dir, f"{filename}_{current_number}.tiff")

    yield from bps.mv(camera.tiff.file_name, filename)
    yield from bps.mv(camera.tiff.file_number, current_number)

    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Attempt {attempt}] Capturing → {filepath}")
            yield from bps.mv(acquire_signal, 1)  # Triggers a single image
            yield from bps.sleep(1)

            # Wait for file to appear
            wait_for_file(filepath)

            print(f"✓ Image saved at {filepath}")
            break  # Exit retry loop if successful

        except TimeoutError:
            print(f"--Timeout waiting for image at {filepath}")
            if attempt == max_retries:
                print(f"--Failed after {max_retries} attempts")
            else:
                print("↻ Retrying acquisition...")
                yield from bps.mv(acquire_signal, 0)
                yield from bps.sleep(0.5)
    
    print("\n--- Unstaging camera ---")
    yield from bps.unstage(camera)

    yield from bps.mv(motor, 0.0)
    yield from bps.close_run()

    file_path_container.clear()
    file_path_container.append(f"{filepath}")

    return print(f"{filepath}")

In [6]:
import tifffile

def convert_image_format(input_image_file: str, output_image_file: str):
    arr = tifffile.imread(input_image_file)

    im = Image.fromarray(arr)
    
    im.save(output_image_file, format="PNG")

    return output_image_file

In [7]:
if __name__ == "__main__":
    # Run scan
    try:
        #angle = 42

        print("Starting script")
        # File configuration
        save_dir = '/home/user/tmpData/AI_scan/measurements/'

        # Ensure the directory exists
        os.makedirs(save_dir, exist_ok=True)
        # Then set the path in EPICS

        file_path = None

        camera.tiff.file_path.put(save_dir)
        camera.tiff.file_template.put('%s%s_%d.tiff')

        file_saved = RE(acquire(angle))

        #print(file_path_container[0])

        png_path = file_path_container[0].replace(".tiff", ".png")

        image_path = convert_image_format(file_path_container[0], png_path)

        crop_box = (800, 800, 1600, 1500)  # (left, upper, right, lower)
        with Image.open(png_path) as img:
            cropped = img.crop(crop_box)
            cropped.save(png_path)  # Overwrite or change name if desired

        os.remove(file_path_container[0])


    except KeyboardInterrupt:
        print("\nScan interrupted by user")
        RE.stop()
    except Exception as e:
        print(f"\nError during scan: {e}")
        #RE.stop()

Starting script

--- Staging camera ---


[Attempt 1] Capturing → /home/user/tmpData/AI_scan/measurements/scan_20250814_134355_pos_0.0_shot_619.tiff


✓ Image saved at /home/user/tmpData/AI_scan/measurements/scan_20250814_134355_pos_0.0_shot_619.tiff

--- Unstaging camera ---


/home/user/tmpData/AI_scan/measurements/scan_20250814_134355_pos_0.0_shot_619.tiff
